In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [2]:
IMG_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 10
CHANNELS = 3

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "tomatoImages",
    shuffle = True,
    image_size = IMG_SIZE,
    batch_size = BATCH_SIZE 
)

Found 6576 files belonging to 4 classes.


In [4]:
def test_train_spilit(dataset, train_size = 0.8, test_size = 0.1, val_size = 0.1, shuffle = True, shuffle_size = 10000):
    ds_size = len(dataset)

    if shuffle:
        dataset = dataset.shuffle(shuffle_size, seed = 12)

    train_ds = dataset.take(int(ds_size*train_size))
    val_ds = dataset.skip(int(ds_size*train_size)).take(int(ds_size*val_size))
    test_ds = dataset.skip(int(ds_size*train_size)).skip(int(ds_size*val_size))
                
    return train_ds, val_ds, test_ds

In [5]:
train_ds, val_ds, test_ds = test_train_spilit(dataset)

In [6]:
print(len(train_ds), len(val_ds), len(test_ds))

164 20 22


In [7]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

In [8]:
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMG_SIZE,IMG_SIZE),
    layers.Rescaling(1/255)
])

In [9]:
data_augmentation =  tf.keras.Sequential([
    layers.RandomFlip("horizantal"),
    layers.RandomRotation(0.2),
])

In [10]:
input_shape = (BATCH_SIZE, IMG_SIZE, IMG_SIZE, CHANNELS)
n_classes = 4

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

D:\azin\tomato\code\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ (32, 256, 256, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (32, 254, 254, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (32, 127, 127, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (32, 125, 125, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (32, 62, 62, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (32, 60, 60, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (32, 30, 30, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (32, 28, 28, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (32, 14, 14, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (32, 12, 12, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (32, 6, 6, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (32, 4, 4, 64)              │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (32, 2, 2, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (32, 256)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (32, 64)                    │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (32, 4)                     │             260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 183,812 (718.02 KB)

 Trainable params: 183,812 (718.02 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [13]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=EPOCHS,
)

Epoch 1/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 141s 830ms/step - accuracy: 0.7041 - loss: 0.6650 - val_accuracy: 0.8031 - val_loss: 0.4193
Epoch 2/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 134s 818ms/step - accuracy: 0.8681 - loss: 0.3280 - val_accuracy: 0.9531 - val_loss: 0.1222
Epoch 3/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 132s 806ms/step - accuracy: 0.9325 - loss: 0.1902 - val_accuracy: 0.9203 - val_loss: 0.2038
Epoch 4/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 132s 806ms/step - accuracy: 0.9576 - loss: 0.1205 - val_accuracy: 0.9672 - val_loss: 0.1067
Epoch 5/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 130s 795ms/step - accuracy: 0.9666 - loss: 0.0987 - val_accuracy: 0.9812 - val_loss: 0.0677
Epoch 6/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 131s 801ms/step - accuracy: 0.9769 - loss: 0.0662 - val_accuracy: 0.9859 - val_loss: 0.0403
Epoch 7/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 133s 813ms/step - accuracy: 0.9740 - loss: 0.0786 - val_accuracy: 0.9797 - val_loss: 0.0523
Epoch 8/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 133s 813ms/step - accuracy: 0.9792 -

In [14]:
model.evaluate(test_ds)

22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 419ms/step - accuracy: 0.9929 - loss: 0.0374


[0.037384238094091415, 0.9928977489471436]